In [ ]:
#https://huggingface.co/facebook/wav2vec2-base-960h

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00


In [ ]:
 from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
 import torch
from torch.utils.data import DataLoader, Dataset

from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

 # load model and tokenizer
 processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
 model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

In [ ]:
notebook_login()

## 1) Load data

### 1.1) Load text data

In [ ]:
dataset = load_dataset('danielroncel/dstc2_dialogues')

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df_chat = dataset['train'].to_pandas()
df_chat.head()

,session_id,turn_index,from,transcript,dialog_acts,audio_file,semantics
0,voip-00d76b791d-20130327_010416,0,user,<user><start>,"[{'slots': [], 'act': 'welcomemsg'}]",None,None
1,voip-00d76b791d-20130327_010416,1,system,"<sys>Hello , welcome to the Cambridge restaura...",None,None,None
2,voip-00d76b791d-20130327_010416,2,user,<user>expensive restaurant in the south part o...,"[{'slots': [['slot', 'food']], 'act': 'request'}]",pt344x_0000993_0001219.wav,"{'json': [{'slots': [['pricerange', 'expensive..."
3,voip-00d76b791d-20130327_010416,3,system,<sys>What kind of food would you like?,None,None,None
4,voip-00d76b791d-20130327_010416,4,user,<user>any,"[{'slots': [['name', 'the good luck chinese fo...",pt344x_0001649_0001680.wav,"{'json': [{'slots': [['this', 'dontcare']], 'a..."


### 1.2) Load audio data

In [ ]:
dataset = load_dataset('danielroncel/dstc2_audios')

Generating train split:   0%|          | 0/25516 [00:00<?, ? examples/s]

## 2) Process audio data

### 2.1) Create columns for audio and path

In [ ]:
def extract_audio_and_path(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    paths = [x["path"] for x in examples["audio"]]

    return {'path': paths, 'array': audio_arrays}

In [ ]:
encoded_dataset = dataset.map(extract_audio_and_path, batched=True)
encoded_dataset

Map:   0%|          | 0/25516 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'session_ids', 'path', 'array'],
        num_rows: 25516
    })
})

In [ ]:
encoded_dataset = encoded_dataset.remove_columns(["audio"])
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['session_ids', 'path', 'array'],
        num_rows: 25516
    })
})

In [ ]:
df_audios = encoded_dataset['train'].to_pandas()
df_audios.head()

,session_ids,path,array
0,voip-00d76b791d-20130327_010416,pt344x_0000993_0001219.wav,"[0.000335693359375, 0.000335693359375, 0.00012..."
1,voip-00d76b791d-20130327_010416,pt344x_0001649_0001680.wav,"[0.0001220703125, 0.000152587890625, 0.0003356..."
2,voip-00d76b791d-20130327_010416,pt344x_0002674_0002736.wav,"[0.000152587890625, 0.000152587890625, 0.00027..."
3,voip-00d76b791d-20130327_010416,pt344x_0003533_0003611.wav,"[0.000213623046875, 0.00054931640625, 0.000671..."
4,voip-00d76b791d-20130327_010416,pt344x_0004453_0004531.wav,"[0.000213623046875, 0.000335693359375, 0.00027..."


In [ ]:
del dataset, encoded_dataset

## 3) Create dataset

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
class DSTC2AudioDataset(Dataset):

    def __init__(self, data):
        self.audios = data

    def __len__(self):
        return len(self.audios)

    def __getitem__(self, idx):
        audio = torch.tensor(self.audios['array'].iloc[idx])
        return audio

In [ ]:
audio_dataset = DSTC2AudioDataset(df_audios)
audio_loader = DataLoader(audio_dataset, batch_size=1, shuffle=False)

## 4) Get transcription

In [ ]:
def transcribe_audio(audio):
    input_values = processor(audio, return_tensors="pt", sampling_rate=16000).input_values
    input_values = torch.tensor(input_values).to(device)
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)
    return transcription[0]

In [ ]:
model.to(device)

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [ ]:
transcriptions = list()

for idx, audio in enumerate(audio_loader):

  if idx % 100 == 0:
    print(f"{idx}/{len(audio_loader)} ({idx/len(audio_loader)*100:.2f}%)")

  audio = audio.squeeze().to(device)

  transcription = transcribe_audio(audio)
  transcriptions.append(transcription)

0/25516 (0.00%)


<ipython-input-24-1d63748ccba8>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_values = torch.tensor(input_values).to(device)


100/25516 (0.39%)
200/25516 (0.78%)
300/25516 (1.18%)
400/25516 (1.57%)
500/25516 (1.96%)
600/25516 (2.35%)
700/25516 (2.74%)
800/25516 (3.14%)
900/25516 (3.53%)
1000/25516 (3.92%)
1100/25516 (4.31%)
1200/25516 (4.70%)
1300/25516 (5.09%)
1400/25516 (5.49%)
1500/25516 (5.88%)
1600/25516 (6.27%)
1700/25516 (6.66%)
1800/25516 (7.05%)
1900/25516 (7.45%)
2000/25516 (7.84%)
2100/25516 (8.23%)
2200/25516 (8.62%)
2300/25516 (9.01%)
2400/25516 (9.41%)
2500/25516 (9.80%)
2600/25516 (10.19%)
2700/25516 (10.58%)
2800/25516 (10.97%)
2900/25516 (11.37%)
3000/25516 (11.76%)
3100/25516 (12.15%)
3200/25516 (12.54%)
3300/25516 (12.93%)
3400/25516 (13.32%)
3500/25516 (13.72%)
3600/25516 (14.11%)
3700/25516 (14.50%)
3800/25516 (14.89%)
3900/25516 (15.28%)
4000/25516 (15.68%)
4100/25516 (16.07%)
4200/25516 (16.46%)
4300/25516 (16.85%)
4400/25516 (17.24%)
4500/25516 (17.64%)
4600/25516 (18.03%)
4700/25516 (18.42%)
4800/25516 (18.81%)
4900/25516 (19.20%)
5000/25516 (19.60%)
5100/25516 (19.99%)
5200/25516 (20

## 5) Add transcription to text dataset

In [ ]:
df_audios['transcription'] = transcriptions

## 6) Upload dataset

In [ ]:
df_audios = df_audios.drop('array', axis=1)
df_audios.head()

,session_ids,path,transcription
0,voip-00d76b791d-20130327_010416,pt344x_0000993_0001219.wav,EXPENSIVE RESTAURANT IN THE SOUTH PARLOF TOWN
1,voip-00d76b791d-20130327_010416,pt344x_0001649_0001680.wav,ANY
2,voip-00d76b791d-20130327_010416,pt344x_0002674_0002736.wav,ADDRESS
3,voip-00d76b791d-20130327_010416,pt344x_0003533_0003611.wav,PIBL FOO
4,voip-00d76b791d-20130327_010416,pt344x_0004453_0004531.wav,NK YOU GOOD BYE


In [ ]:
dataset = Dataset.from_pandas(df_audios)
dataset

Dataset({
    features: ['session_ids', 'path', 'transcription'],
    num_rows: 25516
})

In [ ]:
# Antes, eliminar lo que tenemos ahora subido en este dataset
dataset.push_to_hub('danielroncel/dstc2_dialogues_transcription')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/26 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danielroncel/dstc2_dialogues_transcription/commit/1318b38ac70f3b517f552dba7a2cb884efe06a0c', commit_message='Upload dataset', commit_description='', oid='1318b38ac70f3b517f552dba7a2cb884efe06a0c', pr_url=None, pr_revision=None, pr_num=None)